In [46]:
%cd /Users/2708_samridhi/health_ai_project


/Users/2708_samridhi/health_ai_project


/opt/anaconda3/envs/health_ai/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [47]:
import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd


In [48]:
with pdfplumber.open("data/raw/report6.pdf") as pdf:
    for page in pdf.pages:
        print(page.extract_text())


Patient Name: Ananya Sharma
Age: 22 years
Gender: Female
Test Name Result Units Reference Range
-----------------------------------------------------------------------
Hemoglobin 13.6 g/dL 12.0 - 15.0
Total WBC Count 7400 /uL 4000 - 11000
RBC Count 4.7 million/uL 4.0 - 5.2
Platelet Count 240000 /uL 150000 - 450000
Sodium 138 mmol/L 135 - 145
Potassium 4.1 mmol/L 3.5 - 5.0
Total Bilirubin 0.9 mg/dL 0.3 - 1.2


In [49]:
img = Image.open("data/raw/report9.png")
print(pytesseract.image_to_string(img))


METRO HOSPITAL LAB REPORT

Patient Name: Arjun Patel

Age: 35 Gender: Male
Hb: 9.8 g/dL
wec: 14200 /uL.
Platelets: 92000 /uL.
Sodium: 128 mmol/L
Potassium: 5.9 mmol/L

Total Bilirubin: 2.8 mg/dL.



In [1]:
from src.blood_report_parser import parse_report

text = """
Hemoglobin: 13.2 g/dL
Platelet Count: 240000
Female, Age: 27 years
Total Bilirubin: 1.4 mg/dL
"""
parse_report(text)
print(parse_report(text))
a,b=parse_report(text)


({'hemoglobin': 13.2, 'platelets': 240000.0, 'bilirubin_total': 1.4}, 'male')


In [3]:
from src.blood_report_parser import parse_report
from src.model_parameter_interpreter import interpret_parameters

# --- Sample report text ---
report_text = """
Female, Age: 27 years
Hemoglobin: 11.6 g/dL
WBC Count: 8100 /µL
RBC Count: 4.3 million/µL
Platelet Count: 240000 /µL
Sodium: 136 mmol/L
Potassium: 4.9 mmol/L
Total Bilirubin: 1.4 mg/dL
"""

# --- Parse the report ---
params, sex = parse_report(report_text)
print("Extracted parameters:", params)
print("Detected sex:", sex)

# --- Interpret the parameters ---
results = interpret_parameters(params, sex)
for param, info in results.items():
    print(f"{param}: {info['value']} → {info['status']} (range: {info.get('range')})")


Extracted parameters: {'hemoglobin': 11.6, 'wbc_count': 8100.0, 'rbc_count': 4.3, 'platelets': 240000.0, 'sodium': 136.0, 'potassium': 4.9, 'bilirubin_total': 1.4}
Detected sex: female
hemoglobin: 11.6 → low (range: [12.0, 15.5])
wbc_count: 8100.0 → normal (range: [4000, 11000])
rbc_count: 4.3 → normal (range: [4.2, 5.4])
platelets: 240000.0 → normal (range: [150000, 450000])
sodium: 136.0 → normal (range: [135, 145])
potassium: 4.9 → normal (range: [3.5, 5.1])
bilirubin_total: 1.4 → high (range: [0.1, 1.2])


In [2]:
from src.blood_report_parser import parse_report

text = """
Hemoglobin: 13.2 g/dL
Platelet Count: 240000
Female, Age: 27 years
Total Bilirubin: 1.4 mg/dL
"""

results, sex = parse_report(text)
print(f"Results: {results}")
print(f"Sex: {sex}")


Results: {'hemoglobin': 13.2, 'platelets': 240000.0, 'bilirubin_total': 1.4}
Sex: male
